## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import statsmodels
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split

In [2]:
# Make pandas display all columns
pd.set_option('display.max_columns', None)
# Make pandas display all rows
pd.set_option('display.max_rows', None)


# print big numbers in pd.DataFrame (instead of scientific notation)
pd.set_option('display.precision', 12)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set matplotlib title font size
plt.rcParams['axes.titlesize'] = 18

## Define helper functions

In [3]:
def dataframe_info(df):
    '''
    Takes Pandas dataframe (df). Prints number of rows, number of columns, and three sample rows.
    Returns list of numerical columns and list of non-numerical columns.
    '''
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    object_cols  = df.select_dtypes(exclude=np.number).columns.tolist()

    nrows, ncol = df.shape
    nmid = nrows // 2
    
    print(f"Dimensions: {nrows} rows and {ncol} columns")
    print(f"Numeric columns: {len(numeric_cols)}")
    print(f"Object columns: {len(object_cols)}")

    display(df.iloc[[0,nmid,nrows-1]]) # show three rows: first, mid, and last

    return numeric_cols, object_cols

In [4]:
def cleaned_column_names(column_list):
    '''
    Takes list of column names (e.g. from df.columns). 
    Returns list of cleaned column names that can be assigned to DataFrame.
    Usage: df.columns = cleaned_column_names(df.columns)
    '''
    return [name.strip().lower().replace(' ','').replace('-', '_') for name in column_list]

## EDA

In [5]:
df = pd.read_csv('../data/kc_house_data.csv')
df.columns = cleaned_column_names(df.columns)

In [6]:
numeric_cols, object_cols = dataframe_info(df)

Dimensions: 21597 rows and 21 columns
Numeric columns: 15
Object columns: 6


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.00,3,1.00,1180,5650,1.00,NaN,NONE,Average,7 Average,1180,0.0,1955,0.00,98178,47.51,-122.26,1340,5650
10798,2337300370,2/6/2015,175000.00,3,1.00,1030,8395,1.00,NO,NONE,Good,7 Average,1030,0.0,1960,0.00,98023,47.33,-122.34,1370,9380
21596,1523300157,10/15/2014,325000.00,2,0.75,1020,1076,2.00,NO,NONE,Average,7 Average,1020,0.0,2008,0.00,98144,47.59,-122.30,1020,1357


In [7]:
df[object_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           21597 non-null  object
 1   waterfront     19221 non-null  object
 2   view           21534 non-null  object
 3   condition      21597 non-null  object
 4   grade          21597 non-null  object
 5   sqft_basement  21597 non-null  object
dtypes: object(6)
memory usage: 1012.5+ KB


In [8]:
df[numeric_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   price          21597 non-null  float64
 2   bedrooms       21597 non-null  int64  
 3   bathrooms      21597 non-null  float64
 4   sqft_living    21597 non-null  int64  
 5   sqft_lot       21597 non-null  int64  
 6   floors         21597 non-null  float64
 7   sqft_above     21597 non-null  int64  
 8   yr_built       21597 non-null  int64  
 9   yr_renovated   17755 non-null  float64
 10  zipcode        21597 non-null  int64  
 11  lat            21597 non-null  float64
 12  long           21597 non-null  float64
 13  sqft_living15  21597 non-null  int64  
 14  sqft_lot15     21597 non-null  int64  
dtypes: float64(6), int64(9)
memory usage: 2.5 MB


### Investigate and address issues
- Change dtype
    - `sqft_basement`
- Missing data
    - `yr_renovated`
    - `waterfront`
    - `view`


#### Figure out why sqft_basement was an object rather than numeric

In [9]:
## Change sqft_basement to float (force non-numeric values to NaN)
sqft_basement_float = pd.to_numeric(df['sqft_basement'], errors='coerce')

## Find NaNs (rows that contained non-numeric data)
nan_filt = sqft_basement_float.isna()

## Find out what those rows contained in original df
df['sqft_basement'][nan_filt].value_counts()

?    454
Name: sqft_basement, dtype: int64

Should all of those "questionable" basement values be zeroes?

In [10]:
# sqft_basement should equal difference between sqft_living and sqft_above
sqft_basement_computed = df['sqft_living'] - df['sqft_above']

# Make new df to compare computed vs. recorded basement values
basement_df = pd.DataFrame(dict(
    sqft_basement_float = sqft_basement_float.fillna(0),
    sqft_basement_computed = sqft_basement_computed
    ))

In [11]:
# find out what the computed column contains in the NaN rows... Are they actually zeroes?
basement_df.loc[nan_filt].describe()

,sqft_basement_float,sqft_basement_computed
count,454.00,454.00
mean,0.00,285.82
std,0.00,450.95
min,0.00,0.00
25%,0.00,0.00
50%,0.00,0.00
75%,0.00,560.00
max,0.00,2620.00


In [12]:
# Maybe we should use the computed values?
# In the other rows (without NaNs), does the recorded value always equal the computed value?
notnan_filt = (~nan_filt) & (basement_df['sqft_basement_float'] != basement_df['sqft_basement_computed'])
basement_df.loc[notnan_filt].shape[0]

0

Let's save the computed values to use later, instead of the original recorded values.

In [13]:
df['sqft_basement_computed'] = basement_df['sqft_basement_computed']

In [14]:
# drop the sqft_basement object column, since we're convinced that it is wrong
df.drop(columns=['sqft_basement'], inplace=True)

Let's also create a simple `has_basement` feature, if the `sqft_basement_computed` if greater than 0

In [15]:
df['has_basement'] = (df['sqft_basement_computed'] > 0).astype(int)

#### Determine how to use `yr_renovated`

In [16]:
df['yr_renovated'].describe()

count   17755.00
mean       83.64
std       399.95
min         0.00
25%         0.00
50%         0.00
75%         0.00
max      2015.00
Name: yr_renovated, dtype: float64

In [17]:
df['yr_renovated'].loc[ df['yr_renovated'] > 0 ].describe()

count    744.00
mean    1995.93
std       15.60
min     1934.00
25%     1987.00
50%     2000.00
75%     2007.25
max     2015.00
Name: yr_renovated, dtype: float64

There are tons of missing rows for `yr_renovated` and most rows have values of 0. We can't be certain that any of those houses have been renovated.

Even when there are four-digit year values in `yr_renovated`, lots of them are not even recent!

So, let's make a simple feature `renovated` that captures whether the house was renovated recently (in last 5 years)

In [18]:
df['yr_renovated_missing'] = (df['yr_renovated'].isna()).astype(int)

In [19]:
df['yr_renovated'].fillna(0, inplace=True)

In [20]:
df['sold_dt'] = pd.to_datetime(df['date'])
df['sold_year'] = pd.DatetimeIndex(df['sold_dt']).year
df['sold_month'] = pd.DatetimeIndex(df['sold_dt']).month

In [21]:
df['renovated'] = ((df['sold_year'] - df['yr_renovated']) <= 5).astype(int)

In [22]:
df['renovated'].value_counts()

0    21433
1      164
Name: renovated, dtype: int64

#### Also create a house age column

There are obvious issues with individual values (e.g., -1) that will need to be removed as outliers later

In [23]:
df['house_age'] = df['sold_year'] - df['yr_built']

In [24]:
df['house_age'].describe()

count   21597.00
mean       43.32
std        29.38
min        -1.00
25%        18.00
50%        40.00
75%        63.00
max       115.00
Name: house_age, dtype: float64

#### Fill NaN values in `view` column

In [25]:
df['view'].value_counts(dropna=False)

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
NaN             63
Name: view, dtype: int64

In [26]:
df['view_missing'] = (df['view'].isna()).astype(int)

In [27]:
df['view_missing'].value_counts()

0    21534
1       63
Name: view_missing, dtype: int64

In [28]:
df['view'].fillna('NONE', inplace=True)

In [29]:
df[['view', 'view_missing']].value_counts()

view       view_missing
NONE       0               19422
AVERAGE    0                 957
GOOD       0                 508
FAIR       0                 330
EXCELLENT  0                 317
NONE       1                  63
dtype: int64

#### Fill NaN values in `waterfront` column

In [30]:
df['waterfront'].value_counts(dropna=False)

NO     19075
NaN     2376
YES      146
Name: waterfront, dtype: int64

There are many NaNs in `waterfront`. It's likely that most or all of them should be 'NO', but we can't be certain. So, let's make a `waterfront_missing` flag to keep track of these, so we can evaluate them later.

In [31]:
df['waterfront_missing'] = (df['waterfront'].isna()).astype(int)

In [32]:
df['waterfront_missing'].value_counts()

0    19221
1     2376
Name: waterfront_missing, dtype: int64

In [33]:
df['waterfront'].fillna('NO', inplace=True)

In [34]:
df[['waterfront', 'waterfront_missing']].value_counts()

waterfront  waterfront_missing
NO          0                     19075
            1                      2376
YES         0                       146
dtype: int64

#### Verify that we've addressed NaNs and dtypes

In [35]:
numeric_cols, object_cols = dataframe_info(df)

Dimensions: 21597 rows and 30 columns
Numeric columns: 24
Object columns: 6


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basement_computed,has_basement,yr_renovated_missing,sold_dt,sold_year,sold_month,renovated,house_age,view_missing,waterfront_missing
0,7129300520,10/13/2014,221900.00,3,1.00,1180,5650,1.00,NO,NONE,Average,7 Average,1180,1955,0.00,98178,47.51,-122.26,1340,5650,0,0,0,2014-10-13,2014,10,0,59,0,1
10798,2337300370,2/6/2015,175000.00,3,1.00,1030,8395,1.00,NO,NONE,Good,7 Average,1030,1960,0.00,98023,47.33,-122.34,1370,9380,0,0,0,2015-02-06,2015,2,0,55,0,0
21596,1523300157,10/15/2014,325000.00,2,0.75,1020,1076,2.00,NO,NONE,Average,7 Average,1020,2008,0.00,98144,47.59,-122.30,1020,1357,0,0,0,2014-10-15,2014,10,0,6,0,0


In [36]:
df[numeric_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      21597 non-null  int64  
 1   price                   21597 non-null  float64
 2   bedrooms                21597 non-null  int64  
 3   bathrooms               21597 non-null  float64
 4   sqft_living             21597 non-null  int64  
 5   sqft_lot                21597 non-null  int64  
 6   floors                  21597 non-null  float64
 7   sqft_above              21597 non-null  int64  
 8   yr_built                21597 non-null  int64  
 9   yr_renovated            21597 non-null  float64
 10  zipcode                 21597 non-null  int64  
 11  lat                     21597 non-null  float64
 12  long                    21597 non-null  float64
 13  sqft_living15           21597 non-null  int64  
 14  sqft_lot15              21597 non-null

In [37]:
df[object_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        21597 non-null  object        
 1   waterfront  21597 non-null  object        
 2   view        21597 non-null  object        
 3   condition   21597 non-null  object        
 4   grade       21597 non-null  object        
 5   sold_dt     21597 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 1012.5+ KB


#### Check for duplicate `id` records and decide how to handle

In [38]:
print(f"Number of rows:\t\t{len(df['id'])}")
print(f"Number unique:\t\t{len(df['id'].unique())}")
print(f"Number duplicates:\t{sum(df['id'].duplicated() == True)}")

Number of rows:		21597
Number unique:		21420
Number duplicates:	177


In [41]:
dup_filt = df.sort_values(by=['sold_year', 'sold_month']).duplicated() == True